In [1]:
! pip install "unstructured[all-docs]"

zsh:1: /Users/maymach09/Documents/GenAI09/MacOS/RAG2_Unstructured/tenv/bin/pip: bad interpreter: /Users/maymach09/Documents/GenAI/macbook/GenAI/RAG2/tenv/bin/python3.11: no such file or directory


In [2]:
!pip install "unstructured[s3]"



zsh:1: /Users/maymach09/Documents/GenAI09/MacOS/RAG2_Unstructured/tenv/bin/pip: bad interpreter: /Users/maymach09/Documents/GenAI/macbook/GenAI/RAG2/tenv/bin/python3.11: no such file or directory


In [3]:
import nltk

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/maymach09/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
!pip install poppler-utils
!pip install pandoc

zsh:1: /Users/maymach09/Documents/GenAI09/MacOS/RAG2_Unstructured/tenv/bin/pip: bad interpreter: /Users/maymach09/Documents/GenAI/macbook/GenAI/RAG2/tenv/bin/python3.11: no such file or directory
zsh:1: /Users/maymach09/Documents/GenAI09/MacOS/RAG2_Unstructured/tenv/bin/pip: bad interpreter: /Users/maymach09/Documents/GenAI/macbook/GenAI/RAG2/tenv/bin/python3.11: no such file or directory


In [6]:
from unstructured.partition.pdf import partition_pdf

In [8]:
!pip install torch

zsh:1: /Users/maymach09/Documents/GenAI09/MacOS/RAG2_Unstructured/tenv/bin/pip: bad interpreter: /Users/maymach09/Documents/GenAI/macbook/GenAI/RAG2/tenv/bin/python3.11: no such file or directory


In [11]:
fname = "data/SoftwareRequirementsSpecification.pdf"

elements = partition_pdf(filename=fname,
                         infer_table_structure=True,
                         strategy='hi_res',
           )

tables = [el for el in elements if el.category == "Table"]

print(tables[0].text)
print(tables[0].metadata.text_as_html)

ModuleNotFoundError: No module named 'torch'

In [ ]:
!brew install poppler

zsh:1: command not found: brew
